# **اكتشاف نعاس السائق باستخدام CNN-LSTM**
يهدف هذا النموذج إلى تحليل تسلسل الصور من فيديو السائق لتحديد ما إذا كان في حالة **يقظة** أو **نعاس**.

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# وظيفة لتحميل البيانات وإنشاء تسلسل زمني من الصور
def load_data(data_path, img_size=(64, 64), sequence_length=10):
    X, y = [], []
    labels = {'alert': 0, 'drowsy': 1}  # تصنيف الحالات

    for category in os.listdir(data_path):
        class_path = os.path.join(data_path, category)
        if not os.path.isdir(class_path):
            continue
        
        frames = []
        for img_name in sorted(os.listdir(class_path)):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size) / 255.0
            frames.append(img)

            if len(frames) == sequence_length:
                X.append(frames)
                y.append(labels[category])
                frames = frames[1:]  # التمرير عبر النوافذ الزمنية
        
    X, y = np.array(X), np.array(y)
    return X, y


## **تحميل البيانات**
يتم هنا تحميل البيانات من المسار المحدد، حيث يجب تعديل `data_path` ليشير إلى موقع البيانات لديك.

In [ ]:

# **ضع هنا مسار البيانات الفعلي**
data_path = "/path/to/dataset"  
X, y = load_data(data_path)

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## **بناء نموذج CNN-LSTM**
يتم بناء النموذج باستخدام **CNN لاستخراج الميزات** ثم **LSTM لمعالجة التتابع الزمني**.

In [ ]:

# **بناء نموذج CNN-LSTM**
model = Sequential([
    TimeDistributed(Conv2D(32, (3,3), activation='relu', padding='same'), input_shape=(10, 64, 64, 3)),
    TimeDistributed(MaxPooling2D((2,2))),
    TimeDistributed(Conv2D(64, (3,3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2,2))),
    TimeDistributed(Flatten()),

    LSTM(64, return_sequences=True),
    LSTM(32),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# **إعداد النموذج للتدريب**
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


## **تدريب النموذج**
يتم تدريب النموذج باستخدام البيانات التي تم تحميلها سابقًا.

In [ ]:

# **تدريب النموذج**
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


## **تقييم النموذج**
يتم هنا تقييم دقة النموذج على بيانات الاختبار.

In [ ]:

# **تقييم النموذج**
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")
